# METHODOLOGY TO GET MORE GEOGRAPHICAL GRANULARITY

In [15]:
from scipy.cluster.vq import kmeans2, whiten
import geopandas as gpd
from shapely import geometry as geom
import numpy as np
import pandas as pd
import math
from shapely.ops import cascaded_union
from geovoronoi import voronoi_regions_from_coords, points_to_coords

### 1.- From Landscan, clip country extent and convert from raster to vector (points)

In [16]:
gdf = gpd.read_file('points_population.shp')

### 2,. Remove points with population = 0

In [17]:
gdf = gdf[gdf['VALUE']!=0]

### 3.- Create a new column with a list fo range(0,population) and then explode that column to get a point for each person

In [18]:
def rang(x):
    l = [*range(0,np.round(x/100).astype(int))]
    return l
gdf['list'] = np.array(list(map(rang,(np.round(gdf['VALUE'].values)).astype(int))))

C:\Users\ibra1001\Anaconda3\envs\qgis\lib\site-packages\ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


In [19]:
exploded = pd.DataFrame(gdf).explode('list')

### 4.- Load original geo_keys and demographics

In [20]:
boundaries = gpd.read_file('MBR_CL_R2020_Comunas.shp')

In [21]:
boundaries['ADMINCODE'] = boundaries['ADMINCODE'].astype(int)

In [22]:
demographics = pd.read_excel('MBR_MarketData_2020_CL_Comunas.xlsx')

In [23]:
demographics = demographics.drop(columns=['CTRYCODE','NAME'])

In [24]:
boundaries = boundaries.merge(demographics, on='ADMINCODE')

In [28]:
subset3

,CTRYCODE,NAME,ADMINCODE,geometry,VALUE,P_T,P_PRM,HH_T,HH_SIZE,MALE,...,AGE_F6064,AGE_T6569,AGE_M6569,AGE_F6569,AGE_T7074,AGE_M7074,AGE_F7074,AGE_T75PL,AGE_M75PL,AGE_F75PL
cluster,,,,,,,,,,,,,,,,,,,,,
0,CL,Iquique,1101,POINT (-70.16250 -20.84583),3.0,208695,11.391947,65642,3.179291,104279,...,5362,7966,3817,4149,5088,2477,2611,7644,2958,4686
1,CL,Iquique,1101,POINT (-70.14583 -20.22917),8795.0,208695,11.391947,65642,3.179291,104279,...,5362,7966,3817,4149,5088,2477,2611,7644,2958,4686
2,CL,Iquique,1101,POINT (-70.14583 -20.22083),14547.0,208695,11.391947,65642,3.179291,104279,...,5362,7966,3817,4149,5088,2477,2611,7644,2958,4686
3,CL,Iquique,1101,POINT (-70.12083 -21.11250),41.0,208695,11.391947,65642,3.179291,104279,...,5362,7966,3817,4149,5088,2477,2611,7644,2958,4686
4,CL,Iquique,1101,POINT (-70.13750 -20.18750),13.0,208695,11.391947,65642,3.179291,104279,...,5362,7966,3817,4149,5088,2477,2611,7644,2958,4686
5,CL,Iquique,1101,POINT (-70.13750 -20.22917),11729.0,208695,11.391947,65642,3.179291,104279,...,5362,7966,3817,4149,5088,2477,2611,7644,2958,4686
7,CL,Iquique,1101,POINT (-70.13750 -20.19583),19405.0,208695,11.391947,65642,3.179291,104279,...,5362,7966,3817,4149,5088,2477,2611,7644,2958,4686
8,CL,Iquique,1101,POINT (-70.11250 -20.27083),1805.0,208695,11.391947,65642,3.179291,104279,...,5362,7966,3817,4149,5088,2477,2611,7644,2958,4686
11,CL,Iquique,1101,POINT (-70.13750 -20.21250),7560.0,208695,11.391947,65642,3.179291,104279,...,5362,7966,3817,4149,5088,2477,2611,7644,2958,4686


### 5.- Build Voronoi polygons for each region
- Get points in each region
- K-means to get one cluster/5,000 inh.
- Voronoi for each cluster keeping the most populated point

In [31]:
allpoints = gpd.GeoDataFrame(exploded, geometry='geometry')
j=0
for i in boundaries['ADMINCODE'].unique().tolist():
    print('Region: ',i)    
    region = boundaries[boundaries['ADMINCODE']==i]
    print('Spatial join')
    subset = gpd.sjoin(allpoints, region, how='inner', op='within')
    subset.to_file('subset.shp')
    single = boundaries.explode
    n_clusters = round(subset['VALUE'].count()/50,0)
    if n_clusters < 5:
        n_clusters=5
    coordinates = np.array(list(subset['geometry'].apply(lambda x: list(x.coords[0]))))
    print('K-means')
    x, y = kmeans2(whiten(coordinates), int(n_clusters) , iter = 100, minit = 'points')
    subset['cluster']=y
    subset['duplicated']=subset.duplicated(subset=['geometry','cluster'], keep='first')
    subset2 = subset[subset['duplicated']==False].drop(columns=['index_right'])
    subset2 =subset2.sort_values(['cluster','VALUE'])
    subset3 = subset2.groupby(['cluster']).agg(demo_dict)
    boundary = region.to_crs(epsg=3395)
    subset5 = subset3.set_crs(boundaries.crs)
    subset4 = subset5.to_crs(epsg=3395)
    boundary_shape = cascaded_union(boundary.geometry)
    coords = points_to_coords(subset4.geometry)
    print('Voronoi')
    poly_shapes, pts = voronoi_regions_from_coords(coords, boundary_shape)
    voronoi = gpd.GeoDataFrame(crs='epsg:3395', geometry=list(poly_shapes.values())).to_crs(epsg=4326)
    voronoi2 = gpd.sjoin(voronoi, subset3, how='inner', op='intersects')
    total = voronoi2['VALUE'].sum()

    voronoi2['coeff']=voronoi2['VALUE']/total
    voronoi2['P_T']=voronoi2['coeff']*voronoi2['P_T']
    voronoi2['P_PRM']=voronoi2['coeff']*voronoi2['P_PRM']
    voronoi2['HH_T']=voronoi2['coeff']*voronoi2['HH_T']
    voronoi2['HH_SIZE']=voronoi2['HH_SIZE']
    voronoi2['MALE']=voronoi2['coeff']*voronoi2['MALE']
    voronoi2['FEMALE']=voronoi2['coeff']*voronoi2['FEMALE']
    voronoi2['AGE_T0004']=voronoi2['coeff']*voronoi2['AGE_T0004']
    voronoi2[ 'AGE_M0004']=voronoi2['coeff']*voronoi2[ 'AGE_M0004']
    voronoi2['AGE_F0004']=voronoi2['coeff']*voronoi2['AGE_F0004']
    voronoi2['AGE_T0509']=voronoi2['coeff']* voronoi2['AGE_T0509']
    voronoi2['AGE_M0509']=voronoi2['coeff']* voronoi2['AGE_M0509']
    voronoi2['AGE_F0509']=voronoi2['coeff']*voronoi2['AGE_F0509']
    voronoi2['AGE_T1014']=voronoi2['coeff']*voronoi2['AGE_T1014']
    voronoi2['AGE_M1014']=voronoi2['coeff']* voronoi2['AGE_M1014']
    voronoi2['AGE_F1014']=voronoi2['coeff']*voronoi2['AGE_F1014']
    voronoi2['AGE_T1519']=voronoi2['coeff']*voronoi2['AGE_T1519']
    voronoi2['AGE_M1519']=voronoi2['coeff']* voronoi2['AGE_M1519']
    voronoi2['AGE_F1519']=voronoi2['coeff']*voronoi2['AGE_F1519']
    voronoi2['AGE_T2024']=voronoi2['coeff']*voronoi2['AGE_T2024']
    voronoi2['AGE_M2024']=voronoi2['coeff']*voronoi2['AGE_M2024']
    voronoi2[ 'AGE_F2024']=voronoi2['coeff']*  voronoi2[ 'AGE_F2024']
    voronoi2['AGE_T2529']=voronoi2['coeff']*  voronoi2['AGE_T2529']
    voronoi2['AGE_M2529']=voronoi2['coeff']* voronoi2['AGE_M2529']
    voronoi2['AGE_F2529']=voronoi2['coeff']*voronoi2['AGE_F2529']
    voronoi2['AGE_T3034']=voronoi2['coeff']* voronoi2['AGE_T3034']
    voronoi2['AGE_M3034']=voronoi2['coeff']*voronoi2['AGE_M3034']
    voronoi2['AGE_F3034']=voronoi2['coeff']*voronoi2['AGE_F3034']
    voronoi2['AGE_T3539']=voronoi2['coeff']*voronoi2['AGE_T3539']
    voronoi2['AGE_M3539']=voronoi2['coeff']*voronoi2['AGE_M3539']
    voronoi2['AGE_F3539']=voronoi2['coeff']* voronoi2['AGE_F3539']
    voronoi2['AGE_T4044']=voronoi2['coeff']*voronoi2['AGE_T4044']
    voronoi2['AGE_M4044']=voronoi2['coeff']*voronoi2['AGE_M4044']
    voronoi2['AGE_F4044']=voronoi2['coeff']*voronoi2['AGE_F4044']
    voronoi2['AGE_T4549']=voronoi2['coeff']*voronoi2['AGE_T4549']
    voronoi2['AGE_M4549']=voronoi2['coeff']* voronoi2['AGE_M4549']
    voronoi2['AGE_F4549']=voronoi2['coeff']*voronoi2['AGE_F4549']
    voronoi2['AGE_T5054']=voronoi2['coeff']*voronoi2['AGE_T5054']
    voronoi2['AGE_M5054']=voronoi2['coeff']* voronoi2['AGE_M5054']
    voronoi2['AGE_F5054']=voronoi2['coeff']* voronoi2['AGE_F5054']
    voronoi2['AGE_T5559']=voronoi2['coeff']* voronoi2['AGE_T5559']
    voronoi2['AGE_M5559']=voronoi2['coeff']* voronoi2['AGE_M5559']
    voronoi2['AGE_F5559']=voronoi2['coeff']* voronoi2['AGE_F5559']
    voronoi2['AGE_T6064']=voronoi2['coeff']* voronoi2['AGE_T6064']
    voronoi2['AGE_M6064']=voronoi2['coeff']*voronoi2['AGE_M6064']
    voronoi2['AGE_F6064']=voronoi2['coeff']*voronoi2['AGE_F6064']
    voronoi2['AGE_T6569']=voronoi2['coeff']* voronoi2['AGE_T6569']
    voronoi2['AGE_M6569']=voronoi2['coeff']* voronoi2['AGE_M6569']
    voronoi2['AGE_F6569']=voronoi2['coeff']* voronoi2['AGE_F6569']
    voronoi2['AGE_T7074']=voronoi2['coeff']*voronoi2['AGE_T7074']
    voronoi2['AGE_M7074']=voronoi2['coeff']* voronoi2['AGE_M7074']
    voronoi2['AGE_F7074']=voronoi2['coeff']*voronoi2['AGE_F7074']
    voronoi2['AGE_T75PL']=voronoi2['coeff']*voronoi2['AGE_T75PL']
    voronoi2['AGE_M75PL']=voronoi2['coeff']* voronoi2['AGE_M75PL']
    voronoi2['AGE_F75PL']=voronoi2['coeff']*voronoi2['AGE_F75PL']
   
    voronoi2= voronoi2.drop(columns=['index_right', 'VALUE','coeff'])
    if j==0:
        new_geo_key = voronoi2
    else:
        new_geo_key = gpd.GeoDataFrame( pd.concat( [new_geo_key,voronoi2], ignore_index=True) )
        print(new_geo_key.shape)
    j=j+1

Region:  1101
Spatial join
K-means


C:\Users\ibra1001\Anaconda3\envs\qgis\lib\site-packages\scipy\cluster\vq.py:575: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


Voronoi


C:\Users\ibra1001\Anaconda3\envs\qgis\lib\site-packages\ipykernel_launcher.py:29: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None



Region:  1107
Spatial join
K-means


...

C:\Users\ibra1001\Anaconda3\envs\qgis\lib\site-packages\ipykernel_launcher.py:29: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None



(8775, 56)
Region:  16304
Spatial join
K-means
Voronoi


C:\Users\ibra1001\Anaconda3\envs\qgis\lib\site-packages\ipykernel_launcher.py:29: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None



(8794, 56)
Region:  16305
Spatial join
K-means
Voronoi
(8811, 56)


C:\Users\ibra1001\Anaconda3\envs\qgis\lib\site-packages\ipykernel_launcher.py:29: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None



In [32]:
new_geo_key.to_file('new_geo_key_partial.shp')

### 6.- Check unmatched parts and include them as new geo_keys

In [33]:
boundaries2 = boundaries.explode().reset_index()
boundaries2.to_file('explode.shp')

In [34]:
diference =gpd.overlay(boundaries2, new_geo_key, how='difference')

In [35]:
diference = diference[diference.geom_type == 'Polygon']

In [36]:
diference = diference.drop(columns=['level_0','level_1','NAME','CTRYCODE'])
diference['P_T']=0
diference['P_PRM']=0
diference['HH_T']=0
diference['HH_SIZE']=0
diference['MALE']=0
diference['FEMALE']=0
diference['AGE_T0004']=0
diference[ 'AGE_M0004']=0
diference['AGE_F0004']=0
diference['AGE_T0509']=0
diference['AGE_M0509']=0
diference['AGE_F0509']=0
diference['AGE_T1014']=0
diference['AGE_M1014']=0
diference['AGE_F1014']=0
diference['AGE_T1519']=0
diference['AGE_M1519']=0
diference['AGE_F1519']=0
diference['AGE_T2024']=0
diference['AGE_M2024']=0
diference[ 'AGE_F2024']=0
diference['AGE_T2529']=0
diference['AGE_M2529']=0
diference['AGE_F2529']=0
diference['AGE_T3034']=0
diference['AGE_M3034']=0
diference['AGE_F3034']=0
diference['AGE_T3539']=0
diference['AGE_M3539']=0
diference['AGE_F3539']=0
diference['AGE_T4044']=0
diference['AGE_M4044']=0
diference['AGE_F4044']=0
diference['AGE_T4549']=0
diference['AGE_M4549']=0
diference['AGE_F4549']=0
diference['AGE_T5054']=0
diference['AGE_M5054']=0
diference['AGE_F5054']=0
diference['AGE_T5559']=0
diference['AGE_M5559']=0
diference['AGE_F5559']=0
diference['AGE_T6064']=0
diference['AGE_M6064']=0
diference['AGE_F6064']=0
diference['AGE_T6569']=0
diference['AGE_M6569']=0
diference['AGE_F6569']=0
diference['AGE_T7074']=0
diference['AGE_M7074']=0
diference['AGE_F7074']=0
diference['AGE_T75PL']=0
diference['AGE_M75PL']=0
diference['AGE_F75PL']=0


In [37]:
diference

,ADMINCODE,P_T,P_PRM,HH_T,HH_SIZE,MALE,FEMALE,AGE_T0004,AGE_M0004,AGE_F0004,...,AGE_T6569,AGE_M6569,AGE_F6569,AGE_T7074,AGE_M7074,AGE_F7074,AGE_T75PL,AGE_M75PL,AGE_F75PL,geometry
0,1101,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-70.12958 -20.29565, -70.12961 -20.2..."
6,1404,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-70.15065 -19.84843, -70.15077 -19.8..."
7,1404,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-70.15073 -19.84765, -70.15083 -19.8..."
8,1404,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-70.17172 -19.83159, -70.17177 -19.8..."
9,1404,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-70.17055 -19.83145, -70.17064 -19.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41599,14106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-73.10423 -39.68650, -73.10386 -39.6..."
41600,14106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-73.20966 -39.44717, -73.20991 -39.4..."
41622,16202,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-72.80853 -36.13209, -72.80863 -36.1..."
41623,16202,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((-72.80846 -36.13159, -72.80854 -36.1..."


In [38]:
new_geo_key

,geometry,ADMINCODE,P_T,P_PRM,HH_T,HH_SIZE,MALE,FEMALE,AGE_T0004,AGE_M0004,...,AGE_F6064,AGE_T6569,AGE_M6569,AGE_F6569,AGE_T7074,AGE_M7074,AGE_F7074,AGE_T75PL,AGE_M75PL,AGE_F75PL
0,"POLYGON ((-69.89595 -20.99568, -69.81985 -21.1...",1101,2.840161,0.000155,0.893332,3.179291,1.419148,1.421013,0.189235,0.098068,...,0.072972,0.108410,0.051946,0.056464,0.069243,0.033710,0.035533,0.104028,0.040256,0.063772
1,"POLYGON ((-70.08369 -21.24789, -70.04504 -21.2...",1101,4.260241,0.000233,1.339997,3.179291,2.128722,2.131519,0.283853,0.147101,...,0.109458,0.162616,0.077919,0.084697,0.103865,0.050565,0.053300,0.156042,0.060384,0.095659
2,"POLYGON ((-70.01983 -20.77622, -70.13922 -20.8...",1101,4.260241,0.000233,1.339997,3.179291,2.128722,2.131519,0.283853,0.147101,...,0.109458,0.162616,0.077919,0.084697,0.103865,0.050565,0.053300,0.156042,0.060384,0.095659
3,"POLYGON ((-69.89595 -20.99568, -69.90727 -21.0...",1101,4.260241,0.000233,1.339997,3.179291,2.128722,2.131519,0.283853,0.147101,...,0.109458,0.162616,0.077919,0.084697,0.103865,0.050565,0.053300,0.156042,0.060384,0.095659
4,"POLYGON ((-70.14372 -20.96345, -69.90727 -21.0...",1101,58.223292,0.003178,18.313296,3.179291,29.092535,29.130757,3.879321,2.010384,...,1.495931,2.222414,1.064895,1.157519,1.419488,0.691052,0.728436,2.132580,0.825245,1.307335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8806,"POLYGON ((-72.28750 -36.54041, -72.24761 -36.5...",16305,391.777469,0.021386,134.653686,2.909519,197.240612,194.536857,18.959666,9.045897,...,12.250348,22.164117,11.148818,11.015299,15.988873,8.178025,7.810848,22.798331,10.280946,12.517385
8807,"POLYGON ((-72.19383 -36.55524, -72.19400 -36.5...",16305,35.912935,0.001960,12.343255,2.909519,18.080389,17.832545,1.737969,0.829207,...,1.122949,2.031711,1.021975,1.009736,1.465647,0.749652,0.715994,2.089847,0.942420,1.147427
8808,"POLYGON ((-72.27720 -36.59707, -72.28197 -36.5...",16305,499.516273,0.027267,171.683449,2.909519,251.481780,248.034492,24.173574,11.533519,...,15.619193,28.259249,14.214743,14.044506,20.385814,10.426982,9.958832,29.067872,13.108206,15.959666
8809,"POLYGON ((-72.28750 -36.54041, -72.28750 -36.5...",16305,199.153547,0.010871,68.448957,2.909519,100.263978,98.889569,9.637830,4.598331,...,6.227260,11.266759,5.667316,5.599444,8.127677,4.157163,3.970515,11.589152,5.226147,6.363004


In [39]:
new_geo_key = pd.concat([new_geo_key,diference])

In [40]:
new_geo_key.to_file('new_geo_key2.shp')

Check manually on GIS software to look for unmatched areas

In [41]:
extra=gpd.read_file('extra.shp')

In [45]:
new_geo_key

,geometry,ADMINCODE,P_T,P_PRM,HH_T,HH_SIZE,MALE,FEMALE,AGE_T0004,AGE_M0004,...,AGE_F6064,AGE_T6569,AGE_M6569,AGE_F6569,AGE_T7074,AGE_M7074,AGE_F7074,AGE_T75PL,AGE_M75PL,AGE_F75PL
0,"POLYGON ((-69.89595 -20.99568, -69.81985 -21.1...",1101,2.840161,0.000155,0.893332,3.179291,1.419148,1.421013,0.189235,0.098068,...,0.072972,0.108410,0.051946,0.056464,0.069243,0.033710,0.035533,0.104028,0.040256,0.063772
1,"POLYGON ((-70.08369 -21.24789, -70.04504 -21.2...",1101,4.260241,0.000233,1.339997,3.179291,2.128722,2.131519,0.283853,0.147101,...,0.109458,0.162616,0.077919,0.084697,0.103865,0.050565,0.053300,0.156042,0.060384,0.095659
2,"POLYGON ((-70.01983 -20.77622, -70.13922 -20.8...",1101,4.260241,0.000233,1.339997,3.179291,2.128722,2.131519,0.283853,0.147101,...,0.109458,0.162616,0.077919,0.084697,0.103865,0.050565,0.053300,0.156042,0.060384,0.095659
3,"POLYGON ((-69.89595 -20.99568, -69.90727 -21.0...",1101,4.260241,0.000233,1.339997,3.179291,2.128722,2.131519,0.283853,0.147101,...,0.109458,0.162616,0.077919,0.084697,0.103865,0.050565,0.053300,0.156042,0.060384,0.095659
4,"POLYGON ((-70.14372 -20.96345, -69.90727 -21.0...",1101,58.223292,0.003178,18.313296,3.179291,29.092535,29.130757,3.879321,2.010384,...,1.495931,2.222414,1.064895,1.157519,1.419488,0.691052,0.728436,2.132580,0.825245,1.307335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41599,"POLYGON ((-73.10423 -39.68650, -73.10386 -39.6...",14106,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
41600,"POLYGON ((-73.20966 -39.44717, -73.20991 -39.4...",14106,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
41622,"POLYGON ((-72.80853 -36.13209, -72.80863 -36.1...",16202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
41623,"POLYGON ((-72.80846 -36.13159, -72.80854 -36.1...",16202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [44]:
extra = extra.drop(columns=['NAME','CTRYCODE'])
extra['P_T']=0
extra['P_PRM']=0
extra['HH_T']=0
extra['HH_SIZE']=0
extra['MALE']=0
extra['FEMALE']=0
extra['AGE_T0004']=0
extra[ 'AGE_M0004']=0
extra['AGE_F0004']=0
extra['AGE_T0509']=0
extra['AGE_M0509']=0
extra['AGE_F0509']=0
extra['AGE_T1014']=0
extra['AGE_M1014']=0
extra['AGE_F1014']=0
extra['AGE_T1519']=0
extra['AGE_M1519']=0
extra['AGE_F1519']=0
extra['AGE_T2024']=0
extra['AGE_M2024']=0
extra[ 'AGE_F2024']=0
extra['AGE_T2529']=0
extra['AGE_M2529']=0
extra['AGE_F2529']=0
extra['AGE_T3034']=0
extra['AGE_M3034']=0
extra['AGE_F3034']=0
extra['AGE_T3539']=0
extra['AGE_M3539']=0
extra['AGE_F3539']=0
extra['AGE_T4044']=0
extra['AGE_M4044']=0
extra['AGE_F4044']=0
extra['AGE_T4549']=0
extra['AGE_M4549']=0
extra['AGE_F4549']=0
extra['AGE_T5054']=0
extra['AGE_M5054']=0
extra['AGE_F5054']=0
extra['AGE_T5559']=0
extra['AGE_M5559']=0
extra['AGE_F5559']=0
extra['AGE_T6064']=0
extra['AGE_M6064']=0
extra['AGE_F6064']=0
extra['AGE_T6569']=0
extra['AGE_M6569']=0
extra['AGE_F6569']=0
extra['AGE_T7074']=0
extra['AGE_M7074']=0
extra['AGE_F7074']=0
extra['AGE_T75PL']=0
extra['AGE_M75PL']=0
extra['AGE_F75PL']=0

In [46]:
new_geo_key = pd.concat([new_geo_key,extra])

In [47]:
new_geo_key = new_geo_key.rename(columns={"ADMINCODE": "partition_by"})

In [48]:
new_geo_key['geo_key']=range(10000, 10000 + len(new_geo_key))

In [49]:
new_geo_key.to_file('new_geo_key.shp')

### 7.- Create boundaries and demographics .parquet

In [50]:
new_geo_key

,geometry,partition_by,P_T,P_PRM,HH_T,HH_SIZE,MALE,FEMALE,AGE_T0004,AGE_M0004,...,AGE_T6569,AGE_M6569,AGE_F6569,AGE_T7074,AGE_M7074,AGE_F7074,AGE_T75PL,AGE_M75PL,AGE_F75PL,geo_key
0,"POLYGON ((-69.89595 -20.99568, -69.81985 -21.1...",1101,2.840161,0.000155,0.893332,3.179291,1.419148,1.421013,0.189235,0.098068,...,0.108410,0.051946,0.056464,0.069243,0.033710,0.035533,0.104028,0.040256,0.063772,10000
1,"POLYGON ((-70.08369 -21.24789, -70.04504 -21.2...",1101,4.260241,0.000233,1.339997,3.179291,2.128722,2.131519,0.283853,0.147101,...,0.162616,0.077919,0.084697,0.103865,0.050565,0.053300,0.156042,0.060384,0.095659,10001
2,"POLYGON ((-70.01983 -20.77622, -70.13922 -20.8...",1101,4.260241,0.000233,1.339997,3.179291,2.128722,2.131519,0.283853,0.147101,...,0.162616,0.077919,0.084697,0.103865,0.050565,0.053300,0.156042,0.060384,0.095659,10002
3,"POLYGON ((-69.89595 -20.99568, -69.90727 -21.0...",1101,4.260241,0.000233,1.339997,3.179291,2.128722,2.131519,0.283853,0.147101,...,0.162616,0.077919,0.084697,0.103865,0.050565,0.053300,0.156042,0.060384,0.095659,10003
4,"POLYGON ((-70.14372 -20.96345, -69.90727 -21.0...",1101,58.223292,0.003178,18.313296,3.179291,29.092535,29.130757,3.879321,2.010384,...,2.222414,1.064895,1.157519,1.419488,0.691052,0.728436,2.132580,0.825245,1.307335,10004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41600,"POLYGON ((-73.20966 -39.44717, -73.20991 -39.4...",14106,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60088
41622,"POLYGON ((-72.80853 -36.13209, -72.80863 -36.1...",16202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60089
41623,"POLYGON ((-72.80846 -36.13159, -72.80854 -36.1...",16202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60090
41624,"POLYGON ((-72.80765 -36.13148, -72.80775 -36.1...",16202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60091


In [51]:
new_geo_key[['geo_key','partition_by','geometry']].to_parquet('boundaries.parquet')

C:\Users\ibra1001\Anaconda3\envs\qgis\lib\site-packages\ipykernel_launcher.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  """Entry point for launching an IPython kernel.


In [52]:
centroids = new_geo_key.centroid
new_geo_key['c_x']=centroids.x
new_geo_key['c_y']=centroids.y
new_geo_key = new_geo_key.drop(columns = ['geometry'])

C:\Users\ibra1001\Anaconda3\envs\qgis\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [53]:
new_geo_key.to_parquet('demographics.parquet')

C:\Users\ibra1001\Anaconda3\envs\qgis\lib\site-packages\ipykernel_launcher.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  """Entry point for launching an IPython kernel.


In [29]:
demo_dict = {
'geometry' : 'last',
'ADMINCODE':'last',
'VALUE':'last',
'P_T': 'last',
'P_PRM': 'last',
'HH_T': 'last',
'HH_SIZE': 'mean',
'MALE': 'last',
'FEMALE': 'last',
'AGE_T0004': 'last',
'AGE_M0004': 'last',
'AGE_F0004': 'last',
'AGE_T0509': 'last',
'AGE_M0509': 'last',
'AGE_F0509': 'last',
'AGE_T1014': 'last',
'AGE_M1014': 'last',
'AGE_F1014': 'last',
'AGE_T1519': 'last',
'AGE_M1519': 'last',
'AGE_F1519': 'last',
'AGE_T2024': 'last',
'AGE_M2024': 'last',
'AGE_F2024': 'last',
'AGE_T2529': 'last',
'AGE_M2529': 'last',
'AGE_F2529': 'last',
'AGE_T3034': 'last',
'AGE_M3034': 'last',
'AGE_F3034': 'last',
'AGE_T3539': 'last',
'AGE_M3539': 'last',
'AGE_F3539': 'last',
'AGE_T4044': 'last',
'AGE_M4044': 'last',
'AGE_F4044': 'last',
'AGE_T4549': 'last',
'AGE_M4549': 'last',
'AGE_F4549': 'last',
'AGE_T5054': 'last',
'AGE_M5054': 'last',
'AGE_F5054': 'last',
'AGE_T5559': 'last',
'AGE_M5559': 'last',
'AGE_F5559': 'last',
'AGE_T6064': 'last',
'AGE_M6064': 'last',
'AGE_F6064': 'last',
'AGE_T6569': 'last',
'AGE_M6569': 'last',
'AGE_F6569': 'last',
'AGE_T7074': 'last',
'AGE_M7074': 'last',
'AGE_F7074': 'last',
'AGE_T75PL': 'last',
'AGE_M75PL': 'last',
'AGE_F75PL': 'last',

}